# Gameplan

To get different particle numbers at the same couplings, I will increment different solutions simultaneously! Woah that's crazy.

In [1]:
from solve_rg_eqs import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


l = 8
L = 2*l
print('Ns:')
if l < 5:
    Ns = np.arange(2, 4*l, 2)
elif l < 10:
    Ns = np.arange(2, 2*l+4, 2)
else:
    Ns = np.arange(2, l+4, 2)
print(Ns)
k = np.arange(1, 2*l+1, 2)*0.5*np.pi/l
Gc = 1./np.sum(k)
dg = 0.02/L
g0 = .1*dg
imk = .1*g0
imv = .1*g0

Gf = .2*Gc

gs = np.arange(g0, G_to_g(Gf, k), dg)
qs = np.arange(1/gs[-1], 1/G_to_g(1.2*Gc, k), -10*dg)
print(len(gs))
print(len(qs))

Ns:
[ 2  4  6  8 10 12 14 16 18]
16
4406


In [2]:
def bootstrap_g0_multi(L, g0, kc, imscale_v, final_N=None):
    vs = np.zeros(L)

    Ns = np.arange(2, 4*L, 2)
    if final_N is not None:
        Ns = np.arange(2, final_N+2, 2)
    Ne = Ns[-1]//2
    sols = [None for N in Ns]
    for i, N in enumerate(Ns):
        log('')
        log('Now using {} fermions'.format(N))
        log('')
        n = N//2
        dims = (L, n, n)
        print('Dimensions')
        print(dims)
        # Solving for 2n fermions using extrapolation from previous solution
        if n <= 2:
            force_gs=False
            noise_factors=None
            vars = g0_guess(L, n, n, kc, g0, imscale=imscale_v)
        else:
            # The previous solution matches to roughly the accuracy of the solution
            # for the shared variables
            noise_factors = 10*er*np.ones(len(vars))
            # But we will still need to try random stuff for the 4 new variables
            noise_factors[n-2:n] = 1
            noise_factors[2*n-2:2*n] = 1
            noise_factors[3*n-2:3*n] = 1
            noise_factors[4*n-2:4*n] = 1
        sol = find_root_multithread(vars, kc, g0, dims, imscale_v,
                                    max_steps=MAX_STEPS_1,
                                    use_k_guess=False,
                                    noise_factors=noise_factors,
                                    force_gs=force_gs,
                                    factor=1.2)
        print(vars)
        vars = sol.x
        er = max(abs(rgEqs(vars, kc, g0, dims)))
        log('Error with {} fermions: {}'.format(2*n, er))
        sols[i] = sol
        
        if n%2 == 1:
            # setting up for N divisible by 4 next step
            if n > 1:
                vars = sols[i-1].x # Better to start from last similar case
            n -= 1
            incr = 2
        else:
            incr = 1
        
        if n >= 1:
            es, ws = unpack_vars(vars, n, n)
            vars_guess = g0_guess(L, n+incr, n+incr, kc, g0, imscale=imscale_v)
            esg, wsg = unpack_vars(vars_guess, n+incr, n+incr)
            es = np.append(es, esg[-incr:])
            ws = np.append(ws, wsg[-incr:])
            vars = pack_vars(es, ws)        

    return sols, Ns


def solve_job(prev_vars, kc, g, dims, prev_g):
    sol = root(rgEqs, prev_vars, args=(kc, g, dims), method='lm',
               jac=rg_jac)
    er = np.max(np.abs(sol.fun))
    if er > 10**-6:
        gts = np.linspace(prev_g, g, 10)
        for gt in gts:
            sol = root(rgEqs, prev_vars, args=(kc, gt, dims), method='lm',
                       jac=rg_jac)
            prev_vars = sol.x
    return sol


def solve_job_q(prev_vars, kc, q, dims, prev_q):
    sol = root(rgEqs_q, prev_vars, args=(kc, q, dims), method='lm')
    er = np.max(np.abs(sol.fun))
    if er > 10**-6:
        qts = np.linspace(prev_q, q, 10)
        for gt in qts:
            sol = root(rgEqs_q, prev_vars, args=(kc, qt, dims), method='lm')
            prev_vars = sol.x
    return sol


def solve_g_all(l, Ns, g, kc, prev_g, prev_sols):
    dims = [(l, N//2, N//2) for N in Ns]
    with concurrent.futures.ProcessPoolExecutor(max_workers=CPUS) as executor:
        future_results = [executor.submit(solve_job, ps.x, kc, g, dims[Ni], prev_g) for Ni, ps in enumerate(prev_sols)]
        concurrent.futures.wait(future_results)
        for res in future_results:
            try:
                yield res.result()
            except Exception as e:
                print(e)
                
def solve_q_all(l, Ns, g, kc, prev_q, prev_sols):
    dims = [(l, N//2, N//2) for N in Ns]
    with concurrent.futures.ProcessPoolExecutor(max_workers=CPUS) as executor:
        future_results = [executor.submit(solve_job_q, ps.x, kc, q, dims[Ni], prev_q) for Ni, ps in enumerate(prev_sols)]
        concurrent.futures.wait(future_results)
        for res in future_results:
            try:
                yield res.result()
            except Exception as e:
                print(e)

In [3]:
kim = imk*(-1)**np.arange(1,l+1)
print(kim)
kc = np.concatenate((k, kim))
skip = 10

final_N = Ns[-1]
Ns = np.arange(2, final_N+2, 2)
print(Ns)
prev_sols = [None for N in Ns]
gss = []
varss = []
erss = []

for gi, g in enumerate(gs):
    print('')
    if g == gs[0]:
        prev_sols, Ns = bootstrap_g0_multi(l, g, kc, imscale_v=imv, final_N=final_N)
    else:
        print('g = {}'.format(g))
        g_prev = gs[gi-1]
        for i, s in enumerate(solve_g_all(l, Ns, g, kc, g_prev, prev_sols)):
            prev_sols[i] = s
    for Ni, ps in enumerate(prev_sols):
        N = Ns[Ni]
        print('Error for N = {}'.format(N))
        print(np.max(np.abs(ps.fun)))
    if gi % skip == 0:
        gs += g
        vars_rs = []
        for N in Ns:
            dims = (l, N//2, N//2)
            vars_r, er_r = increment_im_k(prev_sols[Ni].x, dims, g, k, kc[l:], steps=5*L, max_steps=5, force_gs=False)
            vars_rs += [vars_r]
            erss += [er_r]

[-1.25e-05  1.25e-05 -1.25e-05  1.25e-05 -1.25e-05  1.25e-05 -1.25e-05
  1.25e-05]
[ 2  4  6  8 10 12 14 16 18]


Now using 2 fermions

Dimensions
(8, 1, 1)
[ 1.96344722e-01 -1.16246932e-05  0.00000000e+00  0.00000000e+00]
Error with 2 fermions: 2.6720954338088633e-12

Now using 4 fermions

Dimensions
(8, 2, 2)
Bad initial guess. Trying with noise.
g = 0.000125, er = 0.00027198313316330314
1th try at g = 0.000125
Smallest error from last set: 0.00027198313316330314
Noise ranges from -1.4352296672805824e-05 to 1.4985831673829652e-05
[ 1.96344722e-01  1.96344722e-01 -1.16246932e-05 -1.33746932e-05
  1.96347934e-01  1.96347934e-01 -5.24998977e-05  2.75001023e-05]
Error with 4 fermions: 6.585245341415917e-12

Now using 6 fermions

Dimensions
(8, 3, 3)
[ 1.96338137e-01  1.96346485e-01  5.89005250e-01 -1.24985479e-05
 -1.24996109e-05  1.42490796e-05  1.96342311e-01  1.96342311e-01
  0.00000000e+00 -1.49089640e-05 -1.00891947e-05  0.00000000e+00]
Error with 6 fermions: 6.3091647461988034e-12


NameError: name 'dims' is not defined

In [ ]:
final_energies = np.zeros(len(Ns))
for Ni, N in enumerate(Ns):
    print('Removing imaginary parts at N = {}'.format(N))
    dims = (l, N//2, N//2)
    try:
        vars_r, er_r = increment_im_k(prev_sols[Ni].x, dims, g, k, kc[l:], steps=5*L, max_steps=5, force_gs=False)
        varss = np.transpose(np.array([vars_r]))
        energy, _ = calculate_energies(dims, [g], k, varss)
        final_energies[Ni] = energy[0]
        print(er_r)
    except:
        print('Failed at N = {}'.format(N))
        final_energies[Ni] = -1

In [ ]:
print(Ns)
# plt.plot(Ns, final_energies)
plt.figure(figsize=(4,4), dpi=300)
gaps = np.zeros(len(Ns))
G = g_to_G(gs[-1], k)
for Ni, N in enumerate(Ns):
    if Ni == 0:
        em1 = -3*G*np.sum(k**2)
    else:
        em1 = final_energies[Ni-1]
    if N == Ns[-1]:
        ep1 = 4*np.sum(k)-3*G*np.sum(k**2)-4*G*(np.sum(k))**2
    else:
        ep1 = final_energies[Ni+1]
    gaps[Ni] = (ep1+em1 - 2*final_energies[Ni])
plt.plot(Ns, gaps)
plt.ylim(-0.1, 1.0)
plt.xlabel('$N$')
plt.ylabel('$E(N+2)+E(N-2)-2E(N)$')
plt.title('$L$ = {}, $G/G_c$ = {}'.format(L, np.round(G*np.sum(k), 1)))
plt.savefig('quad_gaps_low_g_L_{}.png'.format(L))

In [ ]:
print(np.shape(gs))
print(np.shape(ers))
Gs = g_to_G(gs, k)
for Ni, N in enumerate(Ns):
    if np.max(ers[:, Ni]) < 10**-6:
        plt.plot(energies[:5, Ni])